<a href="https://colab.research.google.com/github/JaswanthMannem/PreTraing/blob/main/3.Data_Packaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install accelerate==0.26.1 datasets==2.16.1 fasttext==0.9.2 jupyter==1.0.0 pandas==2.2.0 pyarrow==15.0.0 sentencepiece==0.1.99 torch==2.1.2 torchaudio==2.1.2 torchvision==0.16.2 tqdm==4.66.1 transformers==4.37.2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import datasets
dataset = datasets.load_dataset(
    "parquet",
    data_files="preprocessed_dataset.parquet",
    split="train"
)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 40474
})


In [4]:
dataset =dataset.shard(num_shards=10,index=0)
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 4048
})


# Tokenization

In [5]:
model_name = "upstage/SOLAR-10.7B-v1.0"

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False
)

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [9]:
tokenizer.tokenize("my name is jaswanth")

['▁my', '▁name', '▁is', '▁j', 'as', 'wan', 'th']

In [11]:
def tokenization(example):
  token = tokenizer.tokenize(example["text"])
  token_ids = tokenizer.convert_tokens_to_ids(token)

  token_ids = [tokenizer.bos_token_id]+token_ids+[tokenizer.eos_token_id]

  example["input_ids"]=token_ids

  example["num_tokens"]=len(token_ids)

  return example


In [29]:
dataset = dataset.map(tokenization, load_from_cache_file=False)
print(dataset)

Map:   0%|          | 0/4048 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'num_tokens'],
    num_rows: 4048
})


In [30]:
sample = dataset[0]

print("text", sample["text"][:30]) #
print("\ninput_ids", sample["input_ids"][:30])
print("\nnum_tokens", sample["num_tokens"])

text In 1793 Zaman Shah, a grandson

input_ids [1, 560, 28705, 28740, 28787, 28774, 28770, 1054, 14886, 23452, 28725, 264, 3487, 1265, 302, 10103, 13179, 23452, 8420, 28712, 4499, 28725, 1747, 264, 6817, 1496, 302, 2288, 296, 298]

num_tokens 644


In [31]:
import numpy as np
np.sum(dataset["num_tokens"])

5113663

# Packing the data

In [32]:
input_ids = np.concatenate(dataset["input_ids"])
print(len(input_ids))

5113663


In [33]:
max_seq_length = 32

In [34]:
total_length = len(input_ids) - len(input_ids) % max_seq_length
print(total_length)

5113632


In [35]:
input_ids = input_ids[:total_length]
print(input_ids.shape)

(5113632,)


In [36]:
input_ids_reshaped = input_ids.reshape(-1, max_seq_length).astype(np.int32)
print(input_ids_reshaped.shape)

(159801, 32)


In [37]:
type(input_ids_reshaped)

numpy.ndarray

In [38]:
input_ids_list = input_ids_reshaped.tolist()
packaged_pretrain_dataset=datasets.Dataset.from_dict(
    {
        "input_ids":input_ids_list
    }
)
print(packaged_pretrain_dataset)

Dataset({
    features: ['input_ids'],
    num_rows: 159801
})


In [39]:
packaged_pretrain_dataset.to_parquet("packaged_pretrain_dataset.parquet")

Creating parquet from Arrow format:   0%|          | 0/160 [00:00<?, ?ba/s]

21093732